# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [7]:
import dask.array as da
import numpy as np
import dask

dask.__version__

'2.5.2'

In [8]:
%%time
arr_np = np.random.normal(0, 1, size = (1000, 300000))
arr_np

Wall time: 14.2 s


array([[ 0.27031297,  0.50776155,  1.6521912 , ..., -0.39927979,
        -1.16654248,  1.02669341],
       [ 0.46980942,  1.69621114,  0.17499064, ..., -0.54195906,
         0.708207  ,  0.9968609 ],
       [ 0.04741525, -0.92665989, -0.36993933, ..., -0.56737443,
        -0.66373472, -1.05478067],
       ...,
       [-0.29801987, -0.14355434,  1.20703496, ..., -0.40217591,
         0.41643898, -1.7240146 ],
       [ 1.08085972, -1.70938938,  1.08258402, ...,  0.99920268,
         1.71850537,  0.81235542],
       [-0.59258977, -1.83212929, -0.22837009, ...,  0.45727153,
        -1.47908463,  1.56685402]])

In [9]:
%%time
arr_da = da.random.normal(0, 1, size = (1000, 300000),
                      chunks = (1000, 30000))
arr_da

Wall time: 0 ns


dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 30000), chunktype=numpy.ndarray>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [10]:
%%time
np.power(arr_np, 2).sum()

Wall time: 17.8 s


299988661.4579253

In [11]:
%%time
# без компьюта ничего не считается, только строится алгоритм
da.power(arr_da, 2).sum().compute()

Wall time: 12.7 s


299996616.3379248

3. Визуализируйте граф вычислений для задачи 2.

## Лабораторная работа 11

In [12]:
import dask.array as da
import h5py
import numpy as np

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [13]:
data_f  = h5py.File('./11_dask_array_data/minutes_n_ingredients_full.hdf5', 'r')
list(data_f.keys())

['recipe']

In [31]:
recipe = da.from_array(data_f['/recipe'], chunks=(100_000, 3))
recipe

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [16]:
%%time
recipe[:, 1:].mean(axis=0).compute()

Wall time: 119 ms


array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [17]:
recipe2 = da.from_array(data_f['/recipe'], chunks=(500_000, 3))
recipe3 = da.from_array(data_f['/recipe'], chunks=(10_000, 3))
recipe4 = da.from_array(data_f['/recipe'], chunks=(100_000, 1))
recipe5 = da.from_array(data_f['/recipe'], chunks=(100_000, 5))
recipe6 = da.from_array(data_f['/recipe'], chunks=(100_000, 100_000))

In [18]:
%%time
recipe2[:, 1:].mean(axis=0).compute()

Wall time: 112 ms


array([1004.20805176,    5.4198008 ])

In [19]:
%%time
recipe3[:, 1:].mean(axis=0).compute()

Wall time: 468 ms


array([1004.20805176,    5.4198008 ])

In [20]:
%%time
recipe4[:, 1:].mean(axis=0).compute()

Wall time: 224 ms


array([1004.20805176,    5.4198008 ])

In [21]:
%%time
recipe5[:, 1:].mean(axis=0).compute()

Wall time: 106 ms


array([1004.20805176,    5.4198008 ])

In [22]:
%%time
recipe6[:, 1:].mean(axis=0).compute()

Wall time: 96.1 ms


array([1004.20805176,    5.4198008 ])

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [23]:
#mask = recipe[:, 1] < recipe[:, 1].median(axis=0)
#mask = recipe[:, 1] < da.median(recipe[:, 1], axis=0)
mask = recipe[:, 1] < np.median(recipe[:, 1], axis=0)
recipe[mask].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

In [35]:
import dask; dask.__version__

'2.5.2'

In [34]:
mask = recipe[:, 1] < da.median(recipe[:, 1], axis=0)

AttributeError: module 'dask.array' has no attribute 'median'

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [24]:
unic, n = da.unique(recipe[:,2], return_counts = True)
# unic, n = dask.compute(unic, n)
dict(zip(unic.compute(), n.compute()))

{1: 222071,
 2: 224158,
 3: 229388,
 4: 234948,
 5: 240720,
 6: 244360,
 7: 247181,
 8: 246747,
 9: 246816,
 10: 22430,
 11: 19094,
 12: 15165,
 13: 11640,
 14: 8284,
 15: 6014,
 16: 4145,
 17: 2793,
 18: 1913,
 19: 1279,
 20: 852,
 21: 529,
 22: 346,
 23: 244,
 24: 178,
 25: 107,
 26: 68,
 27: 55,
 28: 33,
 29: 22,
 30: 20,
 31: 13,
 32: 5,
 33: 4,
 34: 3,
 35: 4,
 36: 1,
 37: 2,
 38: 1,
 39: 1,
 40: 2,
 43: 1}

6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [25]:
maxx = recipe[:, 1].max(axis=0).compute()
maxx

2147483647

In [26]:
# отбросила все значения выше квантиля
# q = da.quantile(recipe[:, 1], 0.75)
q = int(np.quantile(recipe[:, 1], 0.75))
recipe[recipe[:, 1] <= q].compute()

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

In [27]:
mask_less = recipe[:, 1] <= q
mask_more = recipe[:, 1] > q
new_time = (recipe[:, 1] - mask_more * recipe[:, 1] + q * mask_more)
#print(new_time.compute())
#print(recipe[:, 1].compute())
new_recipe = da.stack([recipe[:, 0], new_time, recipe[:, 2]], axis=1)
#print(recipe.compute())
new_recipe.compute()

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1910650,      49,       2],
       [ 713836,       0,       9],
       [ 660699,      49,       8]], dtype=int64)

In [ ]:
clip

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [28]:
preference = da.from_array([30, 64])
preference.compute()

array([30, 64])

In [33]:
# ???? L1 = |sum(v1-v2)| v1, v2 - vectors ????
#argmin
perfect_match = abs(recipe[:, 1:] - preference).sum(axis=1).min()
mask = abs(recipe[:, 1:] - preference).sum(axis=1) == perfect_match
recipe[mask].compute()

array([[270566,     30,     33]], dtype=int64)

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [30]:
summ = 0
k = 0
i = 0
with h5py.File('./11_dask_array_data/minutes_n_ingredients_full.hdf5', 'r') as f:
    for _ in range(240):
        data = da.from_array(f['/recipe'][i:i+10000], chunks=(5000, 3))
        i += 10000
        summ += data[:, 1].sum()
        k += data.shape[0]
    res = summ/k
res.compute()

1004.2080517575215